In [ ]:
# Editor appearance set up & Load plot & Calculate index

# Change backend for interactive plot
%matplotlib nbagg
# %matplotlib osx
# %matplotlib tk
# %matplotlib qt

# This line of "print" must exist right after %matplotlib command, 
# otherwise JN will hang on the first import statement after this.
print('Interactive plot activated')


# Extend width of Jupyter Notebook Cell to the size of browser
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))


# Import packages needed

from PIL import Image
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.patches as patches
from matplotlib.widgets import RectangleSelector, PolygonSelector
import numpy as np
from skimage import io, draw
import mplcursors
import IPython.display as Disp
from ipywidgets import widgets
import cv2
from osgeo import gdal
import pickle
from tkinter import Tk
from tkinter.filedialog import askopenfilename, asksaveasfilename
import pandas as pd


# Load image and print size & pre-process

# Use skimage to load multi-layer tiff file
# Tk().withdraw()
image_file = askopenfilename(title='Load image file')
rgb_ir_img = io.imread(image_file)
print("Image Shape: ", rgb_ir_img.shape)

# Extract layers from the multilayer tiff file and do some adjustments
layer_RGB = np.copy(rgb_ir_img[:, :, 0:3])
layer_IR = np.copy(rgb_ir_img[:, :, 3])
layer_mask = np.copy(rgb_ir_img[:, :, 4])

# Change from 16 bit to 8 bit(max 65535 to max 255)
layer_RGB = np.round(layer_RGB / 257)
layer_RGB = layer_RGB.astype(int)

# Set background to black (You can choose background color when exporting orthomosaic from agisoft, 
# the default is white)
layer_RGB[np.where(layer_mask == 0)] = 0
layer_IR[np.where(layer_mask == 0)] = 0



# Load GPS coordinate from file & Calculate pixel location
plot_loc = askopenfilename(title='Load plot location file')
try:
    with open(plot_loc, 'rb') as f:
        plot_names = pickle.load(f)
        plot_vertices_gps = pickle.load(f)
except Exception as e:
    showerror(type(e).__name__, str(e))


# Calculating pixel location from GPS coordinate

ds = gdal.Open(image_file)
gt = ds.GetGeoTransform()
a, b, c, d, e, f = gt

plot_vertices = []
for i in range(len(plot_vertices_gps)):
    one_plot_vertices_gps = plot_vertices_gps[i]
    one_plot_vertices = []
    for vertex in one_plot_vertices_gps:
        xgeo, ygeo = vertex
        
        x = (xgeo - a) / b
        y = (ygeo - d) / f

        one_plot_vertices.append([x, y])
    one_plot_vertices = np.array(one_plot_vertices)
    one_plot_vertices = np.round(one_plot_vertices)
    one_plot_vertices = one_plot_vertices.astype(int)
    plot_vertices.append(one_plot_vertices)

print('Plot location loaded')



# RGB to HSV (H 0-1, S 0-1, V 0-255)

layer_HSV = matplotlib.colors.rgb_to_hsv(layer_RGB)

layer_hue = layer_HSV[:, :, 0]
layer_saturation = layer_HSV[:, :, 1]
layer_value = layer_HSV[:, :, 2]


# Calculate Vegetation Index and display

# Original formula
# DGCI = ((layer_HSV[:, :, 0] - 60) / 60 + (1 - layer_HSV[:, :, 1]) + (1 - layer_HSV[:, :, 2]/255)) / 3

layer_DGCI = (layer_hue * 360 / 60 + (1 - layer_saturation) + (1 - layer_value / 255)) / 3

print('DGCI calculated')

print('\nDone')

In [ ]:
# Set hue range

def show_hue_area(x):
    hue_range = np.array(x)
    hue_mask[np.where((interested_area_hue>hue_range[0]) * (interested_area_hue<hue_range[1]))] = 1
    hue_mask[np.where(np.invert((interested_area_hue>hue_range[0]) * (interested_area_hue<hue_range[1])))] = 0
    out = red * hue_mask * trans + transformed_rgb * (1 - hue_mask * trans)
    ax.imshow(out)
    
    a, b, c = hue_mask.shape
    canopy = hue_mask[:, :, 0].sum()
    canopy_closure = canopy/(a*b)
    canopy_closure = "{:.2%}".format(canopy_closure)
    canopy_closure_text.value = canopy_closure
    
    
def change_trans(x):
    trans = x
    out = red * hue_mask * trans + transformed_rgb * (1 - hue_mask * trans)
    ax.imshow(out)
    
def save_hue_range(b):
    print('Hue range saved!')
    
color_map = 'gist_gray'
all_vertices = np.concatenate(plot_vertices, axis=0)
ul = np.amin(all_vertices, axis=0)
br = np.amax(all_vertices, axis=0)

interested_area_RGB = layer_RGB[ul[1]:br[1], ul[0]:br[0], :]
interested_area_hue = layer_hue[ul[1]:br[1], ul[0]:br[0]] * 360
hue_mask = np.zeros(interested_area_RGB.shape)
trans = 0.5
red = np.ones(hue_mask.shape) * (1, 0, 0)
transformed_rgb = interested_area_RGB / 255



n_bins = 360
fig, ax = plt.subplots(figsize=(5, 5))
test = interested_area_hue.flatten()
ax.hist(test, bins=n_bins)
mean = np.mean(test)
var = np.var(test)
std = np.std(test)
print('Mean:', mean)
print('Variance:', var)
print('Standard Deviation', std)



default_hue_range = [60, 120]
button_layout = widgets.Layout(width='50%')
style = {'description_width': 'initial'}
hue_slider = widgets.FloatRangeSlider(value=default_hue_range, min=0, max=360, step=0.01, description='Hue Range', layout=widgets.Layout(width='70%'), style = {'description_width': 'initial'})
hue_interactive = widgets.interactive(show_hue_area, x=hue_slider)
trans_slider = widgets.FloatSlider(value=0.5, min=0, max=1, description='Mask Transparency', readout_format='.1f', layout=widgets.Layout(width='70%'), style = {'description_width': 'initial'})
trans_interactive = widgets.interactive(change_trans, x=trans_slider)
canopy_closure_text = widgets.Text(value='0', description='Canopy Closure', disabled=True, layout=widgets.Layout(width='40%'), style = {'description_width': 'initial'})


save_hue_button = widgets.Button(description='Save hue boundaries', layout=button_layout)

vbox_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[hue_slider, trans_slider, canopy_closure_text, save_hue_button], layout=vbox_layout)

hbox_layout = widgets.Layout(align_items='center')
out = widgets.Output()
all_widgets = widgets.HBox(children=[out, buttons], layout=hbox_layout)
display(all_widgets)

save_hue_button.on_click(save_hue_range)



with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    show_hue_area(default_hue_range)
    plt.show()

In [ ]:
# Show indices of plots

def show_RGB(b):
    global cb
    if cb is not None:
        cb.remove()
        cb = None
    ax.imshow(interested_area_RGB)
    
def show_IR(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(interested_area_IR, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_DGCI_image(b):
    global cb
    if cb is not None:
        cb.remove()
    axim = ax.imshow(interested_area_DGCI, cmap=plt.get_cmap(color_map))
    cb = fig.colorbar(axim)
    
def show_temp(b):
    if b.description == 'Show Average Temperature':
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_temps[i]) + '℃', ha='center', va='center')
            
        b.description = 'Hide Average Temperature'
    
    elif b.description == 'Hide Average Temperature':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average Temperature'
        plt.show()
    
def show_DGCI(b):
    if b.description == 'Show Average DGCI': 
        for i in range(len(plot_names)):
            one_plot_vertices = plot_vertices[i]
            one_plot_vertices_transformed = one_plot_vertices - ul
            
            polygon = patches.Polygon(one_plot_vertices_transformed, True, facecolor = matplotlib.colors.to_rgba('red', 0.05), edgecolor=matplotlib.colors.to_rgba('orange', 0.5))
            ax.add_patch(polygon)
            text_loc = np.mean(one_plot_vertices_transformed, 0)
            axtx = ax.text(text_loc[0], text_loc[1], plot_names[i] + '\n' + str(avg_DGCIs[i]), ha='center', va='center')
            
        b.description = 'Hide Average DGCI'
    
    elif b.description == 'Hide Average DGCI':
        ax.patches.clear()
        ax.texts.clear()
        b.description = 'Show Average DGCI'
        plt.show()




color_map = 'gist_gray'
all_vertices = np.concatenate(plot_vertices, axis=0)
ul = np.amin(all_vertices, axis=0)
br = np.amax(all_vertices, axis=0)

interested_area_IR = layer_IR[ul[1]:br[1], ul[0]:br[0]]
interested_area_RGB = layer_RGB[ul[1]:br[1], ul[0]:br[0], :]
interested_area_HSV = layer_HSV[ul[1]:br[1], ul[0]:br[0], :]
interested_area_DGCI = layer_DGCI[ul[1]:br[1], ul[0]:br[0]]


# Calculate avg temp & DGCI for each plot
avg_temps = []
avg_DGCIs = []
avg_Rs = []
avg_Gs = []
avg_Bs = []
avg_Hs = []
avg_Ss = []
avg_Vs = []
canopy_closures = []
for i in range(len(plot_names)):
    one_plot_vertices = plot_vertices[i]
    one_plot_vertices_transformed = one_plot_vertices - ul
    
    rr, cc = draw.polygon(one_plot_vertices_transformed[:, 1], one_plot_vertices_transformed[:, 0], interested_area_IR.shape)
    plot_mask = np.zeros(interested_area_IR.shape, np.float)
    plot_mask[rr, cc] = 1
    mask = plot_mask * hue_mask[:, :, 0]
    inds = np.where(mask == 1)
    avg_IR = np.average(interested_area_IR[inds])
    avg_temp = np.round(avg_IR * 0.04 - 273.15, 2)
    avg_temps.append(avg_temp)
    
    avg_DGCI = np.average(interested_area_DGCI[inds])
    avg_DGCI = np.round(avg_DGCI, 2)
    avg_DGCIs.append(avg_DGCI)
    
    avg_R = np.average(interested_area_RGB[:, :, 0][inds])
    avg_R = np.round(avg_R, 2)
    avg_Rs.append(avg_R)
    
    avg_G = np.average(interested_area_RGB[:, :, 1][inds])
    avg_G = np.round(avg_G, 2)
    avg_Gs.append(avg_G)
    
    avg_B = np.average(interested_area_RGB[:, :, 2][inds])
    avg_B = np.round(avg_B, 2)
    avg_Bs.append(avg_B)
    
    avg_H = np.average(interested_area_HSV[:, :, 0][inds]) * 360
    avg_H = np.round(avg_H, 2)
    avg_Hs.append(avg_H)
    
    avg_S = np.average(interested_area_HSV[:, :, 1][inds])
    avg_S = np.round(avg_S, 4)
    avg_Ss.append(avg_S)
    
    avg_V = np.average(interested_area_HSV[:, :, 2][inds])
    avg_V = np.round(avg_V, 2)
    avg_Vs.append(avg_V)
    
    cnp_cls = mask.sum()/plot_mask.sum()
    cnp_cls = "{:.2%}".format(cnp_cls)
    canopy_closures.append(cnp_cls)
    


button_layout = widgets.Layout(width='50%')
show_RGB_button = widgets.Button(description='Show RGB Image', layout=button_layout)
show_IR_button = widgets.Button(description='Show IR Image', layout=button_layout)
show_DGCIImage_button = widgets.Button(description='Show DGCI Image', layout=button_layout)
show_temp_button = widgets.Button(description='Show Average Temperature', layout=button_layout)
show_DGCI_button = widgets.Button(description='Show Average DGCI', layout=button_layout)

vbox_layout = widgets.Layout(align_items='center', width='50%')
buttons = widgets.VBox(children=[show_RGB_button, show_IR_button, show_DGCIImage_button, show_temp_button, show_DGCI_button], layout=vbox_layout)

out = widgets.Output()
hbox_layout = widgets.Layout(align_items='center')
all_widgets = widgets.HBox(children=[out, buttons], layout=hbox_layout)
display(all_widgets)

show_RGB_button.on_click(show_RGB)
show_IR_button.on_click(show_IR)
show_DGCIImage_button.on_click(show_DGCI_image)
show_temp_button.on_click(show_temp)
show_DGCI_button.on_click(show_DGCI)

with out:
    fig, ax = plt.subplots(figsize=(5, 5))
    fig.subplots_adjust(left=0.1, bottom=0.1, right=0.9, top=0.9)
    ax.imshow(interested_area_RGB)
    cb = None
    plt.show()

In [ ]:
# Save to table

df = pd.DataFrame(data=np.array([plot_names, avg_temps, avg_DGCIs, canopy_closures, avg_Rs, avg_Gs, avg_Bs, avg_Hs, avg_Ss, avg_Vs]).transpose(), columns=['Plot Name', 'Avg Temp', 'Avg DGCI', 'Canopy Closure', 'Avg R', 'Avg G', 'Avg B', 'Avg H', 'Avg S', 'Avg V'])
print(df)


def ask_to_save():
    fn = image_file.split('/')[-1].split('.')[0]
    file_name = asksaveasfilename(filetypes=[('csv', '*.csv')], title='Save Indices', initialfile=fn+'_indices')
    if not file_name:
        return
    if not file_name.endswith('.csv'):
        file_name += '.csv'
        
    try:
        df.to_csv(file_name)
        print('Indices saved to', file_name)
    except Exception as e:
        showerror(type(e).__name__, str(e))


# Tk().withdraw()
ask_to_save()